Construct Adjacency Matrix using hvppi prediction results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
get_path = 'drive/MyDrive/Skip-GNN/'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hvppi_preds_r =pd.read_csv(get_path + 'data/IAV/cleaned_hvppi_predictions.csv')
hvppi_preds_r

,Pro1ID,Pro2ID,Score,Interaction,Segment,IAV Strain,human_pname_raw
0,Q92802,P03428,0.161,yes,PB2,PR8,N42L2_HUMAN
1,Q92802,P03431,0.177,yes,PB1,PR8,N42L2_HUMAN
2,Q92802,P03433,0.132,no,PA,PR8,N42L2_HUMAN
3,Q92802,P03452,0.065,no,HA,PR8,N42L2_HUMAN
4,Q92802,P03466,0.068,no,NP,PR8,N42L2_HUMAN
...,...,...,...,...,...,...,...
272086,Q6PEW1,P03437,0.021,no,HA,Aichi,ZCH12_HUMAN
272087,Q6PEW1,P69277,0.049,no,NS1,Aichi,ZCH12_HUMAN
272088,P0CG32,P03437,0.015,no,HA,Aichi,ZCC18_HUMAN
272089,P0CG32,P69277,0.041,no,NS1,Aichi,ZCC18_HUMAN


### Protein List

In [ ]:
iav_p = hvppi_preds_r['Pro2ID'].drop_duplicates() ## Index 0 to 40
human_p = hvppi_preds_r['Pro1ID'].drop_duplicates() ## Index 41 to 15,684

p_list = pd.concat([iav_p, human_p], ignore_index=True).reset_index(drop=True)
p_list

0        P03428
1        P03431
2        P03433
3        P03452
4        P03466
          ...  
15680    Q6NUS8
15681    P12018
15682    Q96IU2
15683    Q6PEW1
15684    P0CG32
Length: 15685, dtype: object

In [ ]:
p_list.to_csv(get_path + 'data/IAV/protein_list.csv', header=['Protein1_ID'])

In [ ]:
protein_list=pd.read_csv(get_path + 'data/IAV/protein_list.csv')
protein_list.columns=['Index', 'Protein1_ID']
protein_list

,Index,Protein1_ID
0,0,P03428
1,1,P03431
2,2,P03433
3,3,P03452
4,4,P03466
...,...,...
15680,15680,Q6NUS8
15681,15681,P12018
15682,15682,Q96IU2
15683,15683,Q6PEW1


In [ ]:
hvppi_preds = hvppi_preds_r.drop(columns = ['Segment', 'IAV Strain', 'human_pname_raw'])
hvppi_preds 

,Pro1ID,Pro2ID,Score,Interaction
0,Q92802,P03428,0.161,yes
1,Q92802,P03431,0.177,yes
2,Q92802,P03433,0.132,no
3,Q92802,P03452,0.065,no
4,Q92802,P03466,0.068,no
...,...,...,...,...
272086,Q6PEW1,P03437,0.021,no
272087,Q6PEW1,P69277,0.049,no
272088,P0CG32,P03437,0.015,no
272089,P0CG32,P69277,0.041,no


### Construct edgelist

In [ ]:
# ### Construct edgelist

# ## Merge hvppi_preds with protein_list
# hvppi_preds_f = hvppi_preds.drop(columns=['Interaction'])
# df_pro1 = pd.merge(hvppi_preds_f, protein_list, left_on=['Pro1ID'], right_on=['Protein1_ID'])
# df_pro2 = pd.merge(hvppi_preds_f, protein_list, left_on=['Pro2ID'], right_on=['Protein1_ID'])
# m_df = pd.merge(df_pro1, df_pro2, on=['Pro1ID', 'Pro2ID', 'Score'])

# ## Final edgelist
# final_edgelist = m_df[['Index_x', 'Index_y', 'Score']] ## [IAV, Human, Score (edge weight)]
# final_edgelist

In [ ]:
# final_edgelist.to_csv(get_path + '/data/IAV/edgelist.txt', index=False, header=None, sep=' ') ## Space-delimetered text file

### Construct Pos and Neg datasets

In [ ]:
pos_df = hvppi_preds[hvppi_preds['Interaction'] == 'yes']
print(len(pos_df))

neg_df = hvppi_preds[hvppi_preds['Interaction'] == 'no']
print(len(neg_df))

48454
223637


In [ ]:
pos_df = hvppi_preds[hvppi_preds['Interaction'] == 'yes']
pos_df = pos_df[pos_df['Score'] >= 0.375] ## 0.99 threshold filter (2872 rows)
pos_df['label'] = 1
pos_df_f = pos_df[['Pro1ID', 'Pro2ID', 'label']].drop_duplicates()
pos_df_f

# pos_df_f.to_csv(get_path + 'data/IAV/positive_edges_threshold.csv', index=False, header=['Protein1_ID', 'Protein2_ID', 'label'])

,Pro1ID,Pro2ID,label
183,Q13761,P03431,1
529,P02751,P03431,1
535,P02751,P03496,1
536,P02751,P06821,1
537,P02751,P03508,1
...,...,...,...
270714,Q8IUE6,P69277,1
270729,Q96KK5,P69277,1
270732,P04908,P69277,1
271248,O60814,P69277,1


In [ ]:
neg_df = hvppi_preds[hvppi_preds['Interaction'] == 'no']
print(len(neg_df))
np.sort(neg_df['Score'].unique())

223637


array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
       0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
       0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
       0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
       0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
       0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
       0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
       0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
       0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
       0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
       0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
       0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
       0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
       0.117, 0.118, 0.119, 0.12 , 0.121, 0.122, 0.123, 0.124, 0.125,
       0.126, 0.127,

In [ ]:
## No filter on negative dataset
neg_df = hvppi_preds[hvppi_preds['Interaction'] == 'no'].drop_duplicates().reset_index(drop=True) ## (223637 rows)
neg_df['label'] = 0
neg_df
# neg_df_f = neg_df[['Pro1ID', 'Pro2ID', 'label']].drop_duplicates()
# neg_df_f
# neg_df_f.to_csv(get_path + 'data/IAV/negative_edges_threshold.csv', index=False, header=['Protein1_ID', 'Protein2_ID', 'label'])

,Pro1ID,Pro2ID,Score,Interaction,label
0,Q92802,P03433,0.132,no,0
1,Q92802,P03452,0.065,no,0
2,Q92802,P03466,0.068,no,0
3,Q92802,P03468,0.063,no,0
4,Q92802,P03485,0.083,no,0
...,...,...,...,...,...
223632,Q6PEW1,P03437,0.021,no,0
223633,Q6PEW1,P69277,0.049,no,0
223634,P0CG32,P03437,0.015,no,0
223635,P0CG32,P69277,0.041,no,0


In [ ]:
'''
Score threshold
0.143 (100%) 223637 negative samples  
0.07 (50%) 147913 samples (~66% of 223, 637 samples) 50p
0.035 (25%) 62181 samples (~28% of 223, 637 samples) 25p
0.017 (12.5%) 18924 samples (~8.5% of 223, 637 samples) 12.5p
0.007 (5%) 4019 samples (~1.8% of 223, 637 samples) 5p
'''

'\nScore threshold\n0.143 (100%) 223637 negative samples  \n0.07 (50%) 147913 samples (~66% of 223, 637 samples)\n0.035 (25%) 62181 samples (~28% of 223, 637 samples)\n0.017 (12.5%) 18924 samples (~8.5% of 223, 637 samples)\n0.007 (5%) 4019 samples (~1.8% of 223, 637 samples)\n'

In [ ]:
### Threshold filter
fn = '5p'
neg_df_t = neg_df[neg_df['Score'] <= 0.007].drop_duplicates().reset_index(drop=True)
neg_df_t = neg_df_t[['Pro1ID', 'Pro2ID', 'label']]
neg_df_t

,Pro1ID,Pro2ID,label
0,P03999,P03468,0
1,Q16613,P03468,0
2,P62952,P03485,0
3,Q9HD36,P03466,0
4,Q9HD36,P03485,0
...,...,...,...
4014,Q2WGJ8,P03437,0
4015,Q2WGJ8,P69277,0
4016,E9PQX1,P03437,0
4017,E9PQX1,P69277,0


In [ ]:
neg_df_t.to_csv(get_path + 'data/IAV/negative_edges_' + fn + '.csv', index=False, header=['Protein1_ID', 'Protein2_ID', 'label'])

### Construct HVPPI Adjacency Matrix

In [ ]:
## Init empty np array
# adj_mat = np.zeros((protein_list.shape[0], protein_list.shape[0]))
# adj_mat

## one-hot encoding
adj_mat = np.eye(protein_list.shape[0]) ## Returns a 2-D array with ones on the diagonal and zeros elsewhere
adj_mat

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
for i in hvppi_preds.index:
   
   p1 = hvppi_preds['Pro1ID'][i]
   p2 = hvppi_preds['Pro2ID'][i]

   ## Map to index
   p1_idx = protein_list[protein_list['Protein1_ID'] == p1]
   p1_idx = p1_idx['Index'].values[0]
   p2_idx = protein_list[protein_list['Protein1_ID'] == p2]
   p2_idx = p2_idx['Index'].values[0]

   ## Append pred score to adj_mat
   adj_mat[p1_idx][p2_idx] = hvppi_preds['Score'][i] ## (human, IAV)
   adj_mat[p2_idx][p1_idx] = hvppi_preds['Score'][i] ## (IAV, human)
   
  #  print(p1_idx, p2_idx)

In [ ]:
adj_mat

array([[1.   , 0.   , 0.   , ..., 0.063, 0.171, 0.241],
       [0.   , 1.   , 0.   , ..., 0.179, 0.231, 0.198],
       [0.   , 0.   , 1.   , ..., 0.093, 0.15 , 0.122],
       ...,
       [0.063, 0.179, 0.093, ..., 1.   , 0.   , 0.   ],
       [0.171, 0.231, 0.15 , ..., 0.   , 1.   , 0.   ],
       [0.241, 0.198, 0.122, ..., 0.   , 0.   , 1.   ]])

In [ ]:
## Convert to sparse matrix
from scipy import sparse
import scipy.sparse as sp

In [ ]:
adj_mat_sparse = sp.csr_matrix(adj_mat, dtype=float)
print(adj_mat_sparse)

  (0, 0)	1.0
  (0, 41)	0.161
  (0, 42)	0.069
  (0, 43)	0.161
  (0, 44)	0.104
  (0, 45)	0.05
  (0, 46)	0.154
  (0, 47)	0.065
  (0, 48)	0.039
  (0, 49)	0.075
  (0, 50)	0.119
  (0, 51)	0.157
  (0, 52)	0.1159999999999999
  (0, 53)	0.221
  (0, 54)	0.1669999999999999
  (0, 55)	0.069
  (0, 56)	0.094
  (0, 57)	0.087
  (0, 58)	0.089
  (0, 59)	0.107
  (0, 60)	0.186
  (0, 61)	0.088
  (0, 62)	0.142
  (0, 63)	0.045
  (0, 64)	0.113
  :	:
  (15683, 10)	0.061
  (15683, 11)	0.073
  (15683, 15)	0.048
  (15683, 23)	0.049
  (15683, 30)	0.021
  (15683, 31)	0.049
  (15683, 15683)	1.0
  (15684, 0)	0.241
  (15684, 1)	0.198
  (15684, 2)	0.122
  (15684, 3)	0.063
  (15684, 4)	0.102
  (15684, 5)	0.045
  (15684, 6)	0.042
  (15684, 7)	0.237
  (15684, 8)	0.129
  (15684, 9)	0.206
  (15684, 10)	0.067
  (15684, 11)	0.053
  (15684, 15)	0.039
  (15684, 17)	0.088
  (15684, 30)	0.015
  (15684, 31)	0.0409999999999999
  (15684, 40)	0.054
  (15684, 15684)	1.0


In [ ]:
## Verify array
edgelist_read = pd.read_csv(get_path + '/data/IAV/edgelist.txt', sep = ' ', header=None)
edgelist_read

,0,1,2
0,41,0,0.161
1,41,1,0.177
2,41,2,0.132
3,41,3,0.065
4,41,4,0.068
...,...,...,...
272086,15684,11,0.053
272087,15684,15,0.039
272088,15684,30,0.015
272089,15684,31,0.041


In [ ]:
## Save adj matrix as sparse matrix
sparse.save_npz(get_path + '/data/IAV/hvppi_pred_adj_matrix_eye.npz', adj_mat_sparse)

In [ ]:
## Verify array

# protein_list[protein_list['Index']==15732] ## P0CG32
# protein_list[protein_list['Index']==0] ## P03428
# hvppi_preds.loc[(hvppi_preds['Score']== 0.241) & (hvppi_preds['Pro1ID'] == 'P0CG32') & (hvppi_preds['Pro2ID'] == 'P03428')]

# protein_list[protein_list['Index']==15730] ## Q96IU2
# protein_list[protein_list['Index']==2] ## P0C0U1
# hvppi_preds.loc[(hvppi_preds['Score']== 0.043) & (hvppi_preds['Pro1ID'] == 'Q96IU2') & (hvppi_preds['Pro2ID'] == 'P0C0U1')]

# protein_list[protein_list['Index']==42] ## Q92802
# protein_list[protein_list['Index']==1] ## P03431
# hvppi_preds.loc[(hvppi_preds['Score']== 0.177) & (hvppi_preds['Pro1ID'] == 'Q92802') & (hvppi_preds['Pro2ID'] == 'P03431')]